# Leaflet cluster map of talk locations

Run this from the _talks/ directory, which contains .md files of all your talks. This scrapes the location YAML field from each .md file, geolocates it with geopy/Nominatim, and uses the getorg library to output data, HTML, and Javascript for a standalone cluster map.

In [1]:
# !pip install getorg --upgrade
import glob
import getorg
from geopy import Nominatim

IPywidgets and ipyleaflet support enabled.


In [2]:
g = glob.glob("*.md")

In [3]:
geocoder = Nominatim()
location_dict = {}
location = ""
permalink = ""
title = ""

ConfigurationError: Using Nominatim with default or sample `user_agent` "geopy/2.3.0" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.

In [4]:

for file in g:
    with open(file, 'r') as f:
        lines = f.read()
        if lines.find('location: "') > 1:
            loc_start = lines.find('location: "') + 11
            lines_trim = lines[loc_start:]
            loc_end = lines_trim.find('"')
            location = lines_trim[:loc_end]
                            
           
        location_dict[location] = geocoder.geocode(location)
        print(location, "\n", location_dict[location])


Berkeley CA, USA 
 Berkeley, Alameda County, California, United States of America
Los Angeles, CA 
 LA, Los Angeles County, California, United States of America
London, UK 
 London, Greater London, England, UK
San Francisco, California 
 SF, California, United States of America


In [5]:
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="../talkmap", hashed_usernames=False)

'Written map to ../talkmap/'